In [12]:
from itertools import product
import pygame
import os

In [13]:
A=[[0,0,0,0,4],
   [0,0,3,0,0],
   [3,2,0,5,0],
   [0,0,1,0,0],
   [0,0,0,0,4]]

A2=[[0,0,0,0,5,0,0,0,0],
   [0,0,0,4,0,0,0,0,0],
   [0,0,0,0,0,0,0,0,0],
   [0,0,0,0,0,0,0,0,0],
   [0,0,0,0,1,0,0,0,0],
   [0,0,0,0,0,0,0,0,0],
   [0,0,0,0,0,0,0,0,0],
   [0,0,0,0,0,8,0,0,0],
   [0,0,0,0,9,0,0,0,0]]

CELL = 30
MARGIN = 2
BG_COLOR = (255, 255, 255)
TEXT_COLOR = (0, 0, 0)
numeros = [
    [0,4,9],[1,3,8],[0,0,1],[0,-4,5],[-1,-3,4]
]

Matrice=A2
combinations = [[a,b,c,d,e,f,g,h,i] for a,b,c,d,e,f,g,h,i in product(range(1,5), repeat=9)]


In [14]:
### Définition de fonctions ###
def extraction_matrice(matrice, forme):
    n = len(matrice)
    if forme == 1:
        # Enlever 1ère ligne et 1ère colonne
        return [row[1:] for row in matrice[1:]]
    elif forme == 2:
        # Enlever 1ère ligne et dernière colonne
        return [row[:-1] for row in matrice[1:]]
    elif forme == 3:
        # Enlever dernière ligne et dernière colonne
        return [row[:-1] for row in matrice[:-1]]
    elif forme == 4:
        # Enlever 1ère colonne et dernière ligne
        return [row[1:] for row in matrice[:-1]]
    return []


def test(matrice,forme,chemin):
    n = len(matrice)
    L=forme[0]
    forme=forme[1:]

    positions_haut_gauche=[[k,0] for k in range(n)]+[[0,k] for k in range(1,n)]
    positions_bas_droite=[[k,n-1] for k in range(n)]+[[n-1,k] for k in range(1,n)]
    positions_haut_droite=[[k,n-1] for k in range(n)]+[[0,k] for k in range(0,n-1)]
    positions_bas_gauche=[[k,0] for k in range(n)]+[[n-1,k] for k in range(1,n)]
    positions = [positions_haut_gauche,positions_haut_droite,positions_bas_droite,positions_bas_gauche]
    
    pos=positions[L-1]
    nbre = [matrice[p][q] for (p, q) in pos]
    #print(nbre)
    valeurs = sorted(set(nbre))      # unique + tri
    if valeurs and valeurs[0] == 0:  # si le premier est zéro
        valeurs.pop(0)
    #print(valeurs)
    
    if len(valeurs) == 1:  # contrainte respectée
        val = valeurs[0]
        # Ajouter au chemin et continuer récursivement avec la matrice réduite
        sous_matrice = extraction_matrice(matrice,L)
        chemin = chemin + [val]
        if n==2:
            if val==2:
                sous_matrice = extraction_matrice(matrice,L)
                return test(sous_matrice,forme,chemin)
            return "erreur"
        elif n==1:
            if val==1:
                return chemin
            return "erreur"
        else:
            return test(sous_matrice,forme,chemin)
    
    if len(valeurs) == 0:
        val=-1
        sous_matrice = extraction_matrice(matrice,L)
        chemin = chemin + [val]
        if n==1:
            return chemin
        return test(sous_matrice,forme,chemin)

    
    return "erreur"


### fonctions d'affichage ###

def dessiner_carre(surface, coord, couleur, dx=0, dy=0, numero=None, font=None):
    """Dessine une cellule carrée et éventuellement un numéro"""
    x, y = coord
    rect_pixel = pygame.Rect((x+dx)*CELL, (y+dy)*CELL, CELL-MARGIN, CELL-MARGIN)
    pygame.draw.rect(surface, couleur, rect_pixel)
    pygame.draw.rect(surface, (0,0,0), rect_pixel, 1)

    if numero is not None and font is not None:
        text_surf = font.render(str(numero), True, TEXT_COLOR)
        text_rect = text_surf.get_rect(center=rect_pixel.center)
        surface.blit(text_surf, text_rect)

def dessiner_L_colle(surface, rect, forme, couleur, dx=0, dy=0):
    xmin, ymin, xmax, ymax = rect
    coords = []

    if forme == 1:
        coords += [(x, ymin-1) for x in range(xmin-1, xmax+1)]
        coords += [(xmin-1, y) for y in range(ymin, ymax+1)]
        new_xmin = xmin-1
        new_ymin = ymin-1
        new_xmax = xmax
        new_ymax = ymax
    elif forme == 2:
        coords += [(x, ymin-1) for x in range(xmin, xmax+2)]
        coords += [(xmax+1, y) for y in range(ymin, ymax+1)]
        new_xmin = xmin
        new_ymin = ymin-1
        new_xmax = xmax+1
        new_ymax = ymax
    elif forme == 3:
        coords += [(x, ymax+1) for x in range(xmin, xmax+2)]
        coords += [(xmax+1, y) for y in range(ymin, ymax+2)]
        new_xmin = xmin
        new_ymin = ymin
        new_xmax = xmax+1
        new_ymax = ymax+1
    elif forme == 4:
        coords += [(x, ymax+1) for x in range(xmin-1, xmax+1)]
        coords += [(xmin-1, y) for y in range(ymin, ymax+2)]
        new_xmin = xmin-1
        new_ymin = ymin
        new_xmax = xmax
        new_ymax = ymax+1

    if surface is not None:
        for (x, y) in coords:
            rect_pixel = pygame.Rect((x+dx)*CELL, (y+dy)*CELL, CELL-MARGIN, CELL-MARGIN)
            pygame.draw.rect(surface, couleur, rect_pixel)
            pygame.draw.rect(surface, (0,0,0), rect_pixel, 1)

    return (new_xmin, new_ymin, new_xmax, new_ymax)

def enregistrer_sequence(sequence, nom_fichier="image.png", numeros=None):
    """
    sequence : liste des formes L [1,2,3,...]
    numeros : liste de [x, y, nbre] à afficher
    """
    n = len(sequence)
    pygame.init()
    font = pygame.font.SysFont(None, CELL-5)

    # Carré central
    cx = cy = 0
    rect = (cx, cy, cx, cy)

    # Calculer rectangle final
    temp_rect = rect
    for forme in reversed(sequence[:-1]):
        temp_rect = dessiner_L_colle(None, temp_rect, forme, couleur=None)

    xmin, ymin, xmax, ymax = temp_rect
    xmin = min(xmin, cx)
    ymin = min(ymin, cy)
    xmax = max(xmax, cx)
    ymax = max(ymax, cy)

    dx = -xmin
    dy = -ymin

    largeur = (xmax - xmin + 1) * CELL
    hauteur = (ymax - ymin + 1) * CELL
    surface = pygame.Surface((largeur, hauteur))
    surface.fill(BG_COLOR)

    # Couleurs pour les L
    couleurs = [(60*i % 255, 90*i % 255, 150*i % 255) for i in range(1, n+1)]

    # Dessiner carré central
    dessiner_carre(surface, (cx, cy), couleurs[-1], dx=dx, dy=dy)

    # Dessiner L concentriques
    temp_rect = (cx, cy, cx, cy)
    for i, forme in enumerate(reversed(sequence[:-1]), start=1):
        temp_rect = dessiner_L_colle(surface, temp_rect, forme, couleurs[i-1], dx=dx, dy=dy)

    # Ajouter les numéros connus
    if numeros is not None:
        for x, y, val in numeros:
            dessiner_carre(surface, (x+dx, y+dy), (200,200,200), numero=val, font=font)

    os.makedirs("Image", exist_ok=True)
    chemin = os.path.join("Image", nom_fichier)
    pygame.image.save(surface, chemin)
    print(f"Image enregistrée : {chemin}")
    pygame.quit()






In [15]:

#print(combinations[0])
Liste=[]
for Forme in combinations:
    chemin=test(Matrice,Forme,[])
    if chemin!="erreur":
        Liste.append([Forme,chemin])

Liste_unique = []
Liste_forme=[]
Global = []

for [f,c] in Liste:
    if c not in Liste_unique:
        Liste_unique.append(c)
        Global.append([f,c])

Liste_unique_updatée = []
Global_updaté = []

for [f,c] in Global:
    if c[-1]==1:
        Liste_unique_updatée.append(c)
        Global_updaté.append([f,c])

Liste_unique_updatée_2 = []
Global_updatée_2 = []

# limites max autorisées pour chaque valeur
limites = {9: 4, 8: 4, 7: 5, 6: 5, 5: 6, 4: 6, 3: 7, 2: 7}

for [f, c] in Global_updaté:
    indices = {x: (c.index(x) if x in c else None) for x in range(2, 10)}

    # Vérifie uniquement les indices présents (pas None)
    conditions_ok = all(
        (indices[val] is None) or (indices[val] <= limite)
        for val, limite in limites.items()
    )

    if conditions_ok:
        Liste_unique_updatée_2.append(c)
        Global_updatée_2.append([f, c])


Liste_unique_updatée_3 = []
Global_updatée_3 = []

for [f,c] in Global_updatée_2:
    c[-2]=2
    Liste_unique_updatée_3.append(c)
    Global_updatée_3.append([f,c])


#Liste_unique_updatée_4 = []
#Global_updatée_4 = []

#for [f,c] in Global_updatée_3:
 #   if ((c[5]==-1) and (c[6]==-1)):
  #      Liste_unique_updatée_4.append(c)
   #     Global_updatée_4.append([f,c])


Liste_unique_updatée_5 = []
Global_updatée_5 = []

for [f,c] in Global_updatée_3:
    if ((f[-2]==3) or (f[-2]==4)):
        Liste_unique_updatée_5.append(c)
        Global_updatée_5.append([f,c])




In [16]:
Liste_unique_updatée_5 = []
Global_updatée_5 = []

for [f,c] in Global_updatée_3:
    if ((f[-2]==3) or (f[-2]==4)):
        Liste_unique_updatée_5.append(c)
        Global_updatée_5.append([f,c])

In [17]:
print(len(Liste_unique_updatée_5))
sum=0
for [Forme,Chemin] in Global_updatée_5:
    sum+=1
    print([Forme,Chemin])
    enregistrer_sequence(Forme, f"puzzle_{sum}.png",numeros=numeros)

15
[[1, 1, 1, 3, 3, 1, 3, 3, 1], [5, 4, -1, 9, 8, -1, -1, 2, 1]]
Image enregistrée : Image/puzzle_1.png
[[1, 1, 3, 1, 3, 1, 3, 3, 1], [5, 4, 9, -1, 8, -1, -1, 2, 1]]
Image enregistrée : Image/puzzle_2.png
[[1, 1, 3, 3, 1, 1, 3, 3, 1], [5, 4, 9, 8, -1, -1, -1, 2, 1]]
Image enregistrée : Image/puzzle_3.png
[[1, 3, 1, 1, 3, 1, 3, 3, 1], [5, 9, 4, -1, 8, -1, -1, 2, 1]]
Image enregistrée : Image/puzzle_4.png
[[1, 3, 1, 3, 1, 1, 3, 3, 1], [5, 9, 4, 8, -1, -1, -1, 2, 1]]
Image enregistrée : Image/puzzle_5.png
[[1, 3, 3, 1, 1, 1, 3, 3, 1], [5, 9, 8, 4, -1, -1, -1, 2, 1]]
Image enregistrée : Image/puzzle_6.png
[[1, 3, 3, 3, 1, 1, 1, 3, 1], [5, 9, 8, -1, 4, -1, -1, 2, 1]]
Image enregistrée : Image/puzzle_7.png
[[2, 2, 2, 2, 4, 4, 4, 4, 1], [5, 4, -1, 8, 9, -1, -1, 2, 1]]
Image enregistrée : Image/puzzle_8.png
[[3, 1, 1, 1, 3, 1, 3, 3, 1], [9, 5, 4, -1, 8, -1, -1, 2, 1]]
Image enregistrée : Image/puzzle_9.png
[[3, 1, 1, 3, 1, 1, 3, 3, 1], [9, 5, 4, 8, -1, -1, -1, 2, 1]]
Image enregistrée : Image/